### Automatic ICGEM data download

http://icgem.gfz-potsdam.de/home

This notebook uses the selenium package and the Chrome webdriver to automatically download geophysical data from ICGEM website by inputting crucial information.

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

def load_driver(website):
    ''' Opens Chrome driver and sets the ICGEM website
    input >
    website:     string   - website where chrome will navigate to
    '''
    #r/Users/admin/Learn_Python2/Hashtag_Lira/Plataforma_Ensino/Automation_Python-Web/chromedriver
    browser = webdriver.Chrome(ChromeDriverManager().install())#executable_path=path)
    browser.get(website)
    time.sleep(2)
    #     browser.find_element(By.NAME, 'Regular grids').click()
    browser.find_element(By.XPATH, '//*[@id="menu"]/a[5]').click()
    time.sleep(2)
    return browser

def get_list_model(website):
    ''' Opens Chrome driver and sets the ICGEM website
    input >
    website:     string   - website where chrome will navigate to
    '''
    #r/Users/admin/Learn_Python2/Hashtag_Lira/Plataforma_Ensino/Automation_Python-Web/chromedriver
    browser = webdriver.Chrome(ChromeDriverManager().install())#executable_path=path)
    browser.get(website)
    time.sleep(2)
    #     browser.find_element(By.NAME, 'Regular grids').click()
    browser.find_element(By.XPATH, '//*[@id="menu"]/a[2]').click()
    time.sleep(5)
    list_model = []
    ind_max = browser.find_element(By.XPATH, '//*[@id="table_div"]/div/div/table/tbody/tr[1]/td[1]').text
    time.sleep(2)
    for i in range(1, int(ind_max)):
        model_name = browser.find_element(By.XPATH,
                                          f'//*[@id="table_div"]/div/div/table/tbody/tr[{i}]/td[2]')
        list_model.append(model_name.text)
    browser.close()
    return list_model
    
def model_type_selection(browser):
    ''' Selects the kind of model as follows:
    
             1- Longtime Model \n
             2- Model from Series \n
             3- Topography related Model \n
             4- Celestial Object Model \n
             5- Topography \n
             
    input >
    browser:            webdriver.Chrome instance
    '''
    dic_tipo = {
        '1': 'Longtime Model',
        '2': 'Model from Series',
        '3': 'Topography related Model',
        '4': 'Celestial Object Model',
        '5': 'Topography',
    }
    #     type_model_window = browser.find_element(By.NAME, 'sel_type')
    type_model_window = browser.find_element(By.XPATH, '//*[@id="model_sel"]/select')
    type_model_window.click()
    tipo = input(""" \n Select the number related to the desired type of model \n
    \t \t 1- Longtime Model \n
    \t \t 2- Model from Series \n
    \t \t 3- Topography related Model \n
    \t \t 4- Celestial Object Model \n
    \t \t 5- Topography \n
    """)
    assert tipo.isdigit(), 'Please select one of the displayed numbers!'
    for model_index in dic_tipo.keys():
        if model_index == tipo:
            tipo = dic_tipo[model_index]
    tags = type_model_window.find_elements_by_tag_name('option')
    for i, tag in enumerate(tags):
        if tag.text == tipo:
            tag.click()
    time.sleep(2)
    
def model_selection(browser, list_model):
    ''' Selects the model
    
    input >
    browser:            webdriver.Chrome instance
    model_name:         string of the model name
    '''
    model_name = input('Download data from which model?')
    while model_name not in list_model:
        print('The input model does not exist')
        model_name = input('Please retype the model for download ').upper()
    #     model_window = browser.find_element(By.CLASS_NAME, 'col')
    model_window = browser.find_element(By.XPATH, '//*[@id="sel_single"]/select')
    model_window.click()
    model_window.send_keys(model_name)
    time.sleep(3)
    
def get_list_functional(browser):
    list_functional = []
    ind_max = len(browser.find_element(By.XPATH, '//*[@id="content"]/form/div[3]/select').find_elements(By.TAG_NAME, 'option'))
    for i in range(1, ind_max):
        func_name = browser.find_element(By.XPATH, f'//*[@id="content"]/form/div[3]/select/option[{i}]').text
        list_functional.append(func_name)
    return list_functional
    
def functional_selection(browser):
    ''' Selects the functional observable to be downloaded
    
    input >
    browser:            webdriver.Chrome instance
    functional:         string of the functional name
    '''
    list_functional = get_list_functional(browser)
    time.sleep(3)
    functional = input(f'''Choose between the following functionals:
    {list_functional}
    ''')
    while functional not in list_functional:
        print('The input functional does not exist')
        functional = input('Data should be from which observable?')
    #     functional = 'gravity_earth'
    #     functional_window = browser.find_element(By.NAME, 'sel_func')
    functional_window = browser.find_element(By.XPATH, '/html/body/div[2]/form/div[3]/select')
    functional_window.click() #'//*[@id="model_sel"]/select'
    functional_window.send_keys(functional)
    time.sleep(3)
    
def grid_selection(browser):
    ''' Sets the area from which the data is related to
    
    input >
    browser:            webdriver.Chrome instance
    '''
    # Left Latitude 
    longitude_esq = input('Enter left longitude limit of the grid...')
    # longitude_esq = str(-75)
    while int(longitude_esq) < -180:
        longitude_esq = input('Reenter left longitude limit of the grid...')
    browser.find_element(By.NAME, 'grid_left').click()
    browser.find_element(By.NAME, 'grid_left').click()
    browser.find_element(By.NAME, 'grid_left').send_keys(Keys.END)
    browser.find_element(By.NAME, 'grid_left').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_left').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_left').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_left').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_left').send_keys(longitude_esq) # '-75'
    time.sleep(2)
    # Right Latitude 
    longitude_dir = input('Enter right longitude limit of the grid...')
    #longitude_dir = str(-32)
    while int(longitude_dir) > 180:
        longitude_dir = input('Reenter right longitude limit of the grid...')
    browser.find_element(By.NAME, 'grid_right').click()
    browser.find_element(By.NAME, 'grid_right').send_keys(Keys.END)
    browser.find_element(By.NAME, 'grid_right').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_right').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_right').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_right').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_right').send_keys(longitude_dir) #'-32'
    time.sleep(2)
    # Upper Longitude
    latitude_acima = input('Enter top latitude limit of the grid...')
    #latitude_acima = str(6)
    while int(latitude_acima) > 90:
        latitude_acima = input('Reenter top latitude limit of the grid...')
    browser.find_element(By.NAME, 'grid_top').click()
    browser.find_element(By.NAME, 'grid_top').send_keys(Keys.END)
    browser.find_element(By.NAME, 'grid_top').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_top').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_top').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_top').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_top').send_keys(latitude_acima) #'6'
    time.sleep(2)
    # Lower Longitude
    latitude_abaixo = input('Enter bottom latitude limit of the grid...')
    #latitude_abaixo = str(-35)
    while int(latitude_abaixo) < -90:
        latitude_abaixo = input('Reenter bottom latitude limit of the grid...')
    browser.find_element(By.NAME, 'grid_bottom').click()
    browser.find_element(By.NAME, 'grid_bottom').send_keys(Keys.END)
    browser.find_element(By.NAME, 'grid_bottom').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_bottom').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_bottom').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_bottom').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_bottom').send_keys(latitude_abaixo) #'-35'
    time.sleep(2)
    # Grid Step
    espacamento = input('Enter grid spacing...')
    browser.find_element(By.NAME, 'grid_step').click()
    browser.find_element(By.NAME, 'grid_step').send_keys(Keys.END)
    browser.find_element(By.NAME, 'grid_step').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_step').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_step').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_step').send_keys(Keys.BACKSPACE)
    browser.find_element(By.NAME, 'grid_step').send_keys(espacamento) #'2'
    time.sleep(2)
    
# def click_start_computation(browser):
#     ''' Clicks on Start Computation button and waits until data is fully calculated
#     for download on the new tab
    
#     input >
#     browser:            webdriver.Chrome instance
#     '''
#     submit = browser.find_elements_by_tag_name('input')[-1]
#     submit.click()
    
#     browser.switch_to.window(browser.window_handles[-1])
#     print(browser.title)
#     print(browser.find_element_by_link_text('Download Grid').text)
#     print(browser.find_element_by_xpath(
#         '/html/body/div[3]/div/div[1]/table/tbody/tr[7]/td/b[1]/a'
#     ).text)
#     input()
#     while str(100) not in browser.find_element_by_id('progress_bar').get_attribute('style'):
#         time.sleep(1)
#     browser.find_element_by_link_text('Download Grid').click()
#     browser.find_element_by_xpath('/html/body/div[3]/div/div[1]/table/tbody/tr[7]/td/b[1]/a').click()

In [2]:
website = "http://icgem.gfz-potsdam.de/home"
list_model = get_list_model(website)
browser = load_driver(website)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/xt/qpy7_yjd7_g89yx6hv6669gw0000gn/T/ipykernel_44240/4118685684.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())#executable_path=path)


====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/xt/qpy7_yjd7_g89yx6hv6669gw0000gn/T/ipykernel_44240/4118685684.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())#executable_path=path)


In [3]:
model_type_selection(browser)

 
 Select the number related to the desired type of model 

    	 	 1- Longtime Model 

    	 	 2- Model from Series 

    	 	 3- Topography related Model 

    	 	 4- Celestial Object Model 

    	 	 5- Topography 

    1


/Users/admin/anaconda/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [4]:
model_selection(browser, list_model)

Download data from which model?EIGEN-6C4


In [5]:
functional_selection(browser)

Choose between the following functionals:
    ['height_anomaly', 'height_anomaly_ell', 'geoid', 'gravity_disturbance', 'gravity_disturbance_sa', 'gravity_anomaly', 'gravity_anomaly_cl', 'gravity_anomaly_sa', 'gravity_anomaly_bg', 'gravity_earth', 'gravity_ell', 'potential_ell', 'gravitation_ell', 'second_r_derivative']
    gravity_ell


In [6]:
grid_selection(browser)

Enter left longitude limit of the grid...-75
Enter right longitude limit of the grid...-32
Enter top latitude limit of the grid...6
Enter bottom latitude limit of the grid...-35
Enter grid spacing...1


In [7]:
submit = browser.find_elements_by_tag_name('input')[-1]
submit.click()
#

/var/folders/xt/qpy7_yjd7_g89yx6hv6669gw0000gn/T/ipykernel_44240/2589856222.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  submit = browser.find_elements_by_tag_name('input')[-1]


In [8]:
browser.switch_to.window(browser.window_handles[-1])
# print(browser.title)
# print(browser.find_element(By.XPATH,
#     '/html/body/div[3]/div/div[1]/table/tbody/tr[7]/td/b[1]/a'
# ).text)

In [9]:
while str(100) not in browser.find_element(By.ID,
                                           'progress_bar').get_attribute('style'):
    time.sleep(1)
browser.find_element(By.XPATH,
                     '/html/body/div[3]/div/div[1]/table/tbody/tr[7]/td/b[1]/a'
                    ).click()

In [10]:
lista = ['coca', 'guaraná', 'pepsi', 'cerveja']